In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
words = ['onomatopoeia', 'football', 'dendrite', 'transformer']

In [7]:
most_similar = {word: glove_wiki_vectors.most_similar(word) for word in words}

In [8]:
pd.DataFrame(most_similar)

,onomatopoeia,football,dendrite,transformer
0,"(onomatopoeic, 0.7537041306495667)","(soccer, 0.8732221722602844)","(axon, 0.6240024566650391)","(substation, 0.7537822723388672)"
1,"(ideogram, 0.6056678891181946)","(basketball, 0.8555637001991272)","(dendritic, 0.558792233467102)","(substations, 0.7127522826194763)"
2,"(colloquialism, 0.6024584174156189)","(league, 0.815336287021637)","(granule, 0.5448510050773621)","(alternator, 0.7009016275405884)"
3,"(exonym, 0.6022687554359436)","(rugby, 0.8007532954216003)","(subpopulation, 0.543938398361206)","(generator, 0.6650993227958679)"
4,"(archaism, 0.5974574089050293)","(hockey, 0.7833694815635681)","(dendrites, 0.5275632739067078)","(voltage, 0.6320236325263977)"
5,"(interjection, 0.5795845985412598)","(club, 0.7730998992919922)","(thallus, 0.5041684508323669)","(pylon, 0.6195249557495117)"
6,"(metonymy, 0.5772469639778137)","(team, 0.7720499634742737)","(acg, 0.4971705675125122)","(electrical, 0.6102921962738037)"
7,"(onomatopoetic, 0.5703136324882507)","(baseball, 0.76057368516922)","(mesenchyme, 0.49398285150527954)","(transformers, 0.6031559109687805)"
8,"(initialism, 0.5680962204933167)","(players, 0.7399718761444092)","(mitochondrion, 0.4875028729438782)","(turbine, 0.5916614532470703)"
9,"(portmanteau, 0.5679948925971985)","(player, 0.7376238703727722)","(tendril, 0.4763730764389038)","(generators, 0.5863012075424194)"


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [9]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [10]:
sims = {f"Similarity between {word1} and {word2}": glove_wiki_vectors.similarity(word1, word2) for word1, word2 in word_pairs}
sims

{'Similarity between coast and shore': 0.70002717,
 'Similarity between clothes and closet': 0.54627603,
 'Similarity between old and new': 0.6432488,
 'Similarity between smart and intelligent': 0.7552733,
 'Similarity between dog and cat': 0.8798075,
 'Similarity between tree and lawyer': 0.076719455}

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [11]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [12]:
test_words_present = {word: word in glove_wiki_vectors for word in test_words}
test_words_present

{'covididiot': False,
 'fomo': False,
 'frenemies': True,
 'anthropause': False,
 'photobomb': False,
 'selfie': False,
 'pxg': False,
 'pacg': False,
 'cct': True,
 'escc': True}

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [13]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [14]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [15]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283238

In [16]:
glove_wiki_vectors.similarity("white", "success")

0.35182396

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [17]:
analogy('man', 'lawyer', 'woman')

man : lawyer :: woman : ?


,Analogy word,Score
0,attorney,0.754815
1,lawyers,0.662468
2,judge,0.634494
3,counsel,0.623383
4,attorneys,0.620947
5,filed,0.611166
6,wife,0.602795
7,plaintiff,0.601512
8,lawsuit,0.590465
9,complaint,0.587618


In [18]:
print(glove_wiki_vectors.similarity('man', 'president'))
glove_wiki_vectors.similarity('woman', 'president')

0.46169153


0.40033475

In [19]:
print(glove_wiki_vectors.similarity('man', 'leader'))
glove_wiki_vectors.similarity('woman', 'leader')

0.5314655


0.40278605

In [20]:
print(glove_wiki_vectors.similarity('conservative', 'hateful'))
glove_wiki_vectors.similarity('liberal', 'hateful')

0.19269513


0.11685433

In [21]:
print(glove_wiki_vectors.similarity('republican', 'evil'))
glove_wiki_vectors.similarity('democrat', 'evil')

0.20948169


0.1343665

In [22]:
print(glove_wiki_vectors.similarity('woman', 'nurturing'))
glove_wiki_vectors.similarity('man', 'nurturing')

0.22584732


0.16296963

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

1. Yes, for one, we can see some political bias in these examples I have explored, where conservative/republican tends to be closer to more negative words. We also see some racial bias in the 'aboriginal-success' and 'white-success' example, and gender-bias in the 'man-leader', 'woman-leader' example. Overall, playing with more examples than these, I found that there are many 
2. I will give two examples:
    - Consider the case that a company is wanting to put together a list of 'positive' news articles. They may run an algorithm that uses these embeddings to determine which articles should be screened in (labeled as positive) or screened out. Lets imagine this algorithm uses some key-words, and determines how similar the words in the article are to them, if the articles relate too closely to these keywords, they will be screened out. We have shown here that 'conservative/republican' articles are more likely to be screened out in this case, this is problematic as this would lead to some political bias of this tool that is not intended.
    - In a similar idea, we will consider a similar tool being used for screening job applications. Imagine we are looking for someone who will be a good leader, it may be reasonable to look for applications where the constituent words are more similar to 'leader', however we have shown here that this has a bias towards men.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [23]:
from sklearn.datasets import fetch_20newsgroups

In [24]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [25]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [26]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [27]:
samp_df = df[:5]
samp_df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast


In [28]:
# Used this code to build my preprocessing pipeline
# Perform sentence and word segmentation
doc = nlp(samp_df['text'][1][:300]) # taking example 1 since it has newline chars
print(f"***** Original doc: ***** \n{doc.text}\n")
# Want to remove special characters, stop words, punctuation
drop_pos = ["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"]
doc = nlp(" ".join([token.text for token in doc if not token.is_stop]))
doc = nlp(" ".join([token.text for token in doc if not token.pos_ in drop_pos]))
print(f"***** Doc without stopwords and symbols: ***** \n{doc.text}\n")
# Want to replace urls, email addresses, numbers to generics
doc = nlp(" ".join([token.text if not token.like_email else 'EMAIL' for token in doc]))
doc = nlp(" ".join([token.text if not token.like_url else 'URL' for token in doc]))
doc = nlp(" ".join([token.text if not token.like_num else 'NUM' for token in doc]))
print(f"***** Doc without emails etc: ***** \n{doc.text}\n")
# Want to perform lemmatization on the resulting token for token in docs
doc = nlp(" ".join([token.lemma_ for token in doc]))
print(f"***** Doc lemmatized: ***** \n{doc.text}\n")

***** Original doc: ***** 



It's not a bad question: I don't have any refs that list this algorithm
either. But thinking about it a bit, it shouldn't be too hard.

1) Take three of the points and find the plane they define as well as
the circle that they lie on (you say you have this algorithm already)

2) Find the center  

***** Doc without stopwords and symbols: ***** 
bad question refs list algorithm thinking bit hard 1 points find plane define circle lie algorithm 2 Find center

***** Doc without emails etc: ***** 
bad question refs list algorithm thinking bit hard NUM points find plane define circle lie algorithm NUM Find center

***** Doc lemmatized: ***** 
bad question ref list algorithm thinking bit hard NUM point find plane define circle lie algorithm NUM Find center



In [29]:
def spacy_preprop(text, drop_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"]):
    result = []
    for token in nlp(text):
        if not token.is_stop and not token.pos_ in drop_pos:
            if token.like_email:
                continue
                # result.append('EMAIL')
            if token.like_url:
                continue
                # result.append('URL')
            if token.like_num:
                continue
                # result.append('NUM')
            else:
                result.append(token.lemma_)
    return " ".join(result)

In [30]:
samp_df['text_pp'] = samp_df['text'].apply(spacy_preprop)
samp_df

C:\Users\mack-\AppData\Local\Temp\ipykernel_25072\3366228882.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samp_df['text_pp'] = samp_df['text'].apply(spacy_preprop)


,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read U.S.C. wonder help U.S.C. provide pa...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,Actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late UPI Foreign Ministry spokesman Ferhat Ata...


In [31]:
df.loc[:, 'text_pp'] = [spacy_preprop(txt) for txt in df['text']]
df.to_csv('text_pp.csv')

In [32]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,Actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late UPI Foreign Ministry spokesman Ferhat Ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,hi like subscribe Leadership Magazine wonder d...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

 - spaCy itself handles the sentence segmentation/tokenization automatically.
 - I then filter out all stop words, this reduces the amount of semantically uninteresting words in each document.
 - Next, I filter out many uninteresting POS' (parts of sentences) such as symbols etc... These do not add too much semantic meaning to the sentence
 - For any tokens that appear to be numbers, emails, or urls, I filter them to a template word. These do not add any meaning and are very much out of distribution for the word embedding model.
    - EDIT: I later revized this, I found that the num, email, and URL appeared to throw of the topic model, so I decided to instead filter them
 - I then perform lemmatization, reducing words to a base form while maintaining the overall semantics. This effectively reduces the vocabularity size.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

I tried 4, 5, 6, 7, and 8 topics. Since it could be argued that the two sports categories may be very similar, with the same for the religion ones, the computer related ones, and the political ones. I found that n_components=5 seemed to produce a reasonable model.

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import mglearn
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df['text_pp'])

In [34]:
n_comp = 5
lda = LatentDirichletAllocation(n_components=n_comp, random_state=42)
document_topics = lda.fit_transform(X)
print(f"LDA comp shape: {lda.components_.shape}")

LDA comp shape: (5, 35814)


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [35]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())
mglearn.tools.print_topics(
    topics=range(n_comp),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
people        god           file          game          israel        
say           jesus         image         team          turkish       
know          believe       use           year          armenian      
think         entry         window        play          jews          
gun           know          program       player        armenians     
time          church        available     good          israeli       
come          file          run           win           turkey        
thing         people        server        season        jewish        
right         think         widget        think         people        
tell          bible         display       hockey        arab          




Based on these results I will label the topics as:

0. Gun control politics
1. Religion
2. Computers
3. Sports
4. Mid-east

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [36]:
document_topics[:5]

array([[0.98412218, 0.00399003, 0.00396903, 0.00394878, 0.00396999],
       [0.19646137, 0.0028694 , 0.78315179, 0.0028782 , 0.01463924],
       [0.69122357, 0.00372062, 0.29772311, 0.00367852, 0.00365418],
       [0.00584942, 0.00583927, 0.7899    , 0.00574763, 0.19266367],
       [0.48039699, 0.00475435, 0.00472693, 0.00469176, 0.50542998]])

So we found:
0. Gun control
1. Computers
2. Gun control + computers
3. Computers
4. Gun control + middle east

These results are relatively reassuring that our model performs well, it appears as if in each case the model seems to be assigning the topic well.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. For context based filters:
    - We get some information about both the items we will recommend, and accumulate some data about the users that may interact with these items.
    - For the items, label them with their features (such as topics, etc...)
    - Using the user data we have and the feature data about our items. We then train a regression model for each user, this can then be used to make predictions about how much a user may be interested in a new, unseen item.
2. For one, recommender systems tend to be biased towards recommending more of the same thing (if a user appears to like something, the recommender will continue to recommend it). This can be a problem as it reduces the diversity of items/options the user has, and may infact reduce their likelihood of exploring new items and therefore revealing to the recommender system a new category of items the user likes. In addition to this they have the potential to cause a phenomenon called 'filter bubbles' where they recommend more and more content that align with the users view, which is particularly dangerous in the context of topics like politics.
3. Transfer learning is the process of taking a pretrained model that has been trained on a dataset that is not specific to your task, and fine-tuning it to be used for a specific task. Typically this pretrained model is trained on a large, general dataset, and then we fine-tune on a dataset that is tailored to our purpose.

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)